# Задание 13

Дан Марковский Процесс Принятия Решений и таблица полезностей `Q`

Нужно подменить ровно одно значение полезности `состояние-действие` так, чтобы жадный алгоритм получил наименьшую дисконтированную награду при коэффициенте дисконтирования $\gamma = 0.9$

BruteForce Emulation: 4/10 баллов

Не догадался улучшить, пробуя не только минимизировать лучшее из значений, но и максимизировать хужшее из значений

In [1]:
import os
from json import load as json_to_obj
from json import dump as obj_to_json

data_path = os.path.join('/tf','shared_data','profi-23', '13') + os.path.sep

mdps_path = data_path + 'mdps.json'
strategy_path = data_path + 'q_tables.json'
sample_path = data_path + 'submit.json'
submission_path = data_path + 'submission.json'


def read_json(path):
    with open(path, 'r') as f:
        json_obj = json_to_obj(f)
    return json_obj


def write_json(path, obj):
    with open(path, 'w') as f:
        obj_to_json(obj, f)

Список словарей содержит данные о каждом МППР, который определяется состояниями, действиями, вероятностями перехода и вознаграждениями.

In [2]:
mdps = read_json(mdps_path)
print(len(mdps))
mdps[0]

1000


{'0': {'0': [{'probability': 1.0, 'next_state': '1', 'reward': 0.33}],
  '1': [{'probability': 1.0, 'next_state': '1', 'reward': -0.64}]},
 '1': {'0': [{'probability': 1.0, 'next_state': '0', 'reward': 0.1}],
  '1': [{'probability': 1.0, 'next_state': '0', 'reward': -0.25}]}}

Словарь определяющий полезность каждого действия, жадная стратегия на его основе используется для управления системой. Формат состояние -> действие -> полезность.


In [3]:
q_tables = read_json(strategy_path)
print(len(q_tables))
q_tables[0]

1000


{'0': {'0': 2.2063543094206666, '1': 1.2363543094206664},
 '1': {'0': 2.0857188784786, '1': 1.7357188784786}}

Список изменений для одной пары состояния-действия каждого МППР, которые необходимо произвести.

In [4]:
sample = read_json(sample_path)
print(len(sample))
sample[0]

1000


{'state': '0', 'action': '0', 'value': -1}

### Baseline
```python
def create_dummy_answer(q_table_):
    # Выбираем первое состояние словаря и
    # присваиваем -1 его самому полезному действию.
    state, actions = next(iter(q_table_.items()))
    max_action = max(actions, key=actions.get)
    return {'state': state, 'action': max_action, 'value': -1}

answers = []
for q_table in q_tables:
    answers.append(create_dummy_answer(q_table))
```

## BOSSline

In [5]:
from random import choices

def calc_reward(model, strategy):
    total = 0
    
    for start_state in strategy.keys():
        coef = 1
        state = start_state
        for step in range(4000):
            best_action = next(iter(strategy[state]))
            
            for action in strategy[state].keys():
                if strategy[state][action] > strategy[state][best_action]:
                    best_action = action

            probs = []
            for jump in model[state][best_action]:
                probs.append(jump['probability'])

            jump = choices(model[state][best_action], weights=probs)[0]
            
            total += coef * jump['reward']
            state = jump['next_state']
            coef *= 0.9
            
    return total


def create_smart_answer(model, strategy):
    best_state = next(iter(strategy))
    best_action = next(iter(strategy[best_state]))
    new_value = strategy[best_state][best_action]
    best_reward = calc_reward(model, strategy)
    
    for state, actions in strategy.items():
        for action, value in actions.items():
            saved = value
            
            strategy[state][action] = -10
            reward = calc_reward(model, strategy)
            strategy[state][action] = saved
            
            if reward < best_reward:
                best_reward = reward
                best_state = state
                best_action = action
                new_value = -10
    return {'state': best_state, 'action': best_action, 'value': new_value}

In [6]:
from tqdm.notebook import tqdm

answers = []
total_tests = len(q_tables)
for i in tqdm(range(total_tests)):
    answers.append(create_smart_answer(mdps[i], q_tables[i]))

  0%|          | 0/1000 [00:00<?, ?it/s]

In [7]:
write_json(submission_path, answers)